# Beskrivelse av datarensing og tidshåndtering for værdata

## Introduksjon
Dette Python-skriptet bearbeider tidligere hentede værdata (nedbør, temperatur, vindhastighet, luftfuktighet) ved å korrigere tidspunktene basert på eventuelle tidsforskyvninger (`timeOffset`) og fjerner unødvendige kolonner. 
Dataene lastes inn, prosesseres, og lagres på nytt.

---

## Funksjoner

- **Les inn data**:  
  Leser værdata fra fire separate CSV-filer ved hjelp av `pandas`.

- **Konverter tidsstempler**:  
  Konverterer kolonnen `referenceTime` fra tekst til korrekt `datetime`-format. Eventuelle feil i konverteringen håndteres ved å sette ugyldige verdier til `NaT`.

- **Justering av tid**:  
  Ekstraherer antall timer fra `timeOffset`-kolonnen ved hjelp av regular expressions og legger dette til `referenceTime` for å justere tidsstemplene.

- **Fjerning av hjelpekollonner**:  
  Etter justeringen fjernes den midlertidige kolonnen `timeOffsetHours` som ble brukt for beregning.

- **Fjerning av uønskede kolonner**:  
  Fjerner kolonner som ikke er relevante for videre analyse, inkludert `level`, `timeResolution`, `timeSeriesId`, `performanceCategory`, `exposureCategory`, og `qualityCode`.

- **Overskriving av CSV-filer**:  
  De oppdaterte datasettene lagres tilbake i sine respektive filer, og eventuelle feil under prosesseringen logges.

---

## Brukte bibliotek

- `pandas` — for å lese inn, manipulere og lagre tabellbasert data.
- `re` (implicit via `str.extract`) — for å trekke ut tall fra tekststrenger ved hjelp av regular expressions.

---

## CSV-filer som behandles

- `temperature_data.csv`
- `precipitation_data.csv`
- `wind_speed_data.csv`
- `humidity_data.csv`

---

# Oppsummering
Dette skriptet sørger for at værdataene har **riktige tidsstempler**, som tar hensyn til eventuelle **forskyvninger**.  
Dette er avgjørende for videre analyse, som f.eks. å sammenligne data på tvers av lokasjoner, modellere tidstrender eller utføre prediktiv analyse.

---

In [1]:
import pandas as pd

# Filstier for datasettene
file_paths = {
    "precipitation_data": "../data/precipitation_data.csv",
    "temperature_data": "../data/temperature_data.csv",
    "wind_speed_data": "../data/wind_speed_data.csv",
    "humidity_data": "../data/humidity_data.csv"
}

# Kolonner som skal slettes
columns_to_delete = ["level", "timeResolution", "timeSeriesId", "performanceCategory", "exposureCategory", "qualityCode"]

# Iterer gjennom hver fil og utfør operasjonene
for dataset_name, file_path in file_paths.items():
    try:
        # Les inn data
        dataset = pd.read_csv(file_path)

        # Konverter referenceTime til datetime
        dataset['referenceTime'] = pd.to_datetime(dataset['referenceTime'], errors='coerce')

        # Hent antall timer fra timeOffset
        dataset['timeOffsetHours'] = dataset['timeOffset'].str.extract(r'PT(\d+)').astype(float)

        # Legg til timeOffset i referenceTime
        dataset['referenceTime'] = dataset['referenceTime'] + pd.to_timedelta(dataset['timeOffsetHours'], unit='h')

        # Slett timeOffsetHours-kolonnen
        dataset.drop(columns=['timeOffsetHours'], inplace=True)

        # Slett spesifiserte kolonner
        dataset.drop(columns=columns_to_delete, inplace=True, errors='ignore')

        # Lagre tilbake til CSV
        dataset.to_csv(file_path, index=False)

        print(f"{dataset_name} behandlet og lagret.")
    except Exception as e:
        print(f"Feil ved behandling av {dataset_name}: {e}")

precipitation_data behandlet og lagret.
temperature_data behandlet og lagret.
wind_speed_data behandlet og lagret.
humidity_data behandlet og lagret.
